In [ ]:
import json, os
import boto
import findspark
# Add the streaming package and initialize
findspark.add_packages(["org.apache.spark:spark-streaming-kafka-0-8_2.11:2.2.0"])
findspark.init()
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import pymongo_spark
pymongo_spark.activate()
from pymongo import MongoClient
from configparser import ConfigParser
from boto.s3.connection import S3Connection
from boto.s3.key import Key
from mypredictor import myStreamPredictor

In [ ]:
def predict(x):
    sqlcontext = pyspark.sql.SQLContext(sc, spark)
    df = sqlcontext.createDataFrame(x)
    prediction = pred.predict(df)
    prediction.show()

In [ ]:
#authenticate
config = ConfigParser()
config.read('../.config/.credentials')
region = config.get('aws', 'region')
AWS_ACCESS_KEY_ID = config.get('aws', 'AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = config.get('aws', 'AWS_SECRET_ACCESS_KEY')
os.environ['AWS_ACCESS_KEY_ID'] = AWS_ACCESS_KEY_ID
os.environ['AWS_SECRET_ACCESS_KEY'] = AWS_SECRET_ACCESS_KEY

In [ ]:
#setting up environment
PERIOD=10
BROKERS='localhost:9092'
TOPIC= 'twitterstream'
duration=100
conf = SparkConf().set("spark.default.paralleism", 1)
spark = pyspark.sql.SparkSession.builder \
            .master("local[4]") \
            .appName('Streamer') \
            .config(conf=conf) \
            .getOrCreate()
sc = spark.sparkContext
#create a streaming context with batch interval 10 sec
ssc = StreamingContext(sc, PERIOD)

In [ ]:
#create stream receivers
stream = KafkaUtils.createDirectStream(
      ssc,
      [TOPIC],
      {
        "metadata.broker.list": BROKERS,
      }
)
tweets = stream.map(lambda x: json.loads(x[1])).map(lambda x: json.loads(x))
    
text = tweets.map(lambda x: {'_id': x['id'], 'text': x['text']})

In [ ]:
#initialize predictor
pred = myStreamPredictor()
#initialize mongodb
client = MongoClient()
collection = client.streams.tweets
collection.drop()

In [ ]:
tweets.count().pprint()
tweets.foreachRDD(lambda x: print('Collection Count is {}'.format(collection.count())))
text.foreachRDD(lambda x: predict(x))
tweets.foreachRDD(lambda x: x.saveToMongoDB('mongodb://localhost:27017/streams.tweets'))


In [ ]:
ssc.start()
ssc.awaitTermination()